In [ ]:
#Importar as bibliotecas e as ferramentas que serão usadas;
import os
import pandas as pd
import panel as pn
import holoviews as hv
import hvplot.pandas
import sqlalchemy
from sqlalchemy import create_engine, Column, String, ForeignKey, func, Integer
from sqlalchemy.orm import declarative_base, sessionmaker
from dotenv import load_dotenv

pn.extension('bokeh')
hv.extension('bokeh')

In [ ]:
#Carregar configurações do .env
load_dotenv() #Retorna True se o arquivo .env for encontrado

In [ ]:
#Inicialização e criação da base que as classes herdarão;
pn.extension()
Base = declarative_base()

In [ ]:
pip install jupyter_bokeh

In [ ]:
#Conexão dinâmica (Usando os dados do .env);
db_user = os.getenv('DB_USER')
db_pass = os.getenv('DB_PASS')
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_port = os.getenv('DB_PORT', '5432')

In [ ]:
#Utilizei psycopg2 para conversar com o PostgreSQL.
DATABASE_URL = f"postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
#Mapeamento das classes utilizando a Biblioteca SQLAlchemy;
class Usuario(Base):
    __tablename__ = 'usuario'
    login = Column(String, primary_key=True)

class Departamento(Base):
    __tablename__ = 'departamento'
    nome_dep = Column(String, primary_key=True)

class Medico(Base):
    __tablename__ = 'medico'
    id = Column(Integer, primary_key=True, autoincrement=True) 
    crm = Column(String, unique=True, nullable=False)         
    login = Column(String, ForeignKey('usuario.login'), unique=True, nullable=False)
    nome_dep = Column(String, ForeignKey('departamento.nome_dep'), nullable=False)

In [ ]:
try:
    # Busca os nomes da tabela Departamento
    deps_query = session.query(Departamento.nome_dep).all()
    lista_departamentos = [d[0] for d in deps_query]
    
    # Aproveite para buscar os logins da tabela Usuario também
    users_query = session.query(Usuario.login).all()
    lista_logins = [u[0] for u in users_query]
except Exception as e:
    print(f"Erro ao acessar o banco: {e}")
    lista_departamentos = []
    lista_logins = []

In [ ]:
#Widgets da Interface;
in_crm = pn.widgets.TextInput(name='CRM', placeholder='Ex: CE123456')
in_login = pn.widgets.Select(name='Selecionar Login do Usuário', options=lista_logins)
in_dep = pn.widgets.Select(name='Departamento', options=lista_departamentos)

input_busca = pn.widgets.TextInput(name='Buscar por CRM', placeholder='Pesquisar...')
alerta = pn.pane.Markdown("", styles={'color': 'red'})
tabela_visualizacao = pn.widgets.DataFrame(pd.DataFrame(), width=700, height=300)

In [ ]:
#Funções de Lógica e mensagens de confirmção;
def atualizar_menu_login():
    # Busca logins reais na tabela Usuario
    logins_existentes = [u.login for u in session.query(Usuario).all()]
    in_login.options = logins_existentes
    
def carregar_dados(event=None):
    atualizar_menu_login()
    query = session.query(Medico)
    if input_busca.value:
        query = query.filter(Medico.crm.ilike(f"%{input_busca.value}%"))
    df = pd.read_sql(query.statement, engine)
    tabela_visualizacao.value = df

def incluir_medico(event):
    try:
        if not in_crm.value or not in_login.value:
            alerta.object = "### Preencha CRM e Login!"
            return
            
        usuario_ja_cadastrado = session.query(Medico).filter_by(login=in_login.value).first()
        if usuario_ja_cadastrado:
            alerta.object = f"### Erro: O usuário '{in_login.value}' já está alocado em {usuario_ja_cadastrado.nome_dep}."
            return

        crm_em_uso = session.query(Medico).filter_by(crm=in_crm.value).first()
        if crm_em_uso:
            alerta.object = f"### Erro: O CRM {in_crm.value} já pertence a outro médico."
            return
            
        # Se passou, inclui normalmente
        novo = Medico(crm=in_crm.value, login=in_login.value, nome_dep=in_dep.value)
        session.add(novo)
        session.commit()
        alerta.object = "### Médico incluído com sucesso!"
        carregar_dados()
        
    except Exception as e:
        session.rollback()
        alerta.object = f"### Erro técnico: {e}"
        
def editar_medico(event):
    medico = session.query(Medico).filter_by(crm=in_crm.value, login=in_login.value).first()
    if medico:
        medico.nome_dep = in_dep.value
        session.commit()
        alerta.object = "### Departamento editado!"
        carregar_dados()
    else:
        alerta.object = "### Médico não encontrado (CRM + Login necessários)."

def remover_medico(event):
    medico = session.query(Medico).filter_by(crm=in_crm.value, login=in_login.value).first()
    if medico:
        session.delete(medico)
        session.commit()
        alerta.object = "### Médico removido!"
        carregar_dados()
    else:
        alerta.object = "### Não foi possível remover."


@pn.depends(input_busca)
def criar_grafico_setores(valor_busca):
    # Busca os dados no banco
    df = pd.read_sql("SELECT nome_dep, COUNT(*) as total FROM medico GROUP BY nome_dep", engine)
    
    if df.empty:
        return pn.pane.Markdown("### Nenhum dado disponível.")
    
    # Criando o gráfico com o estilo da imagem de referência
    return df.hvplot.bar(
        x='nome_dep', 
        y='total', 
        xlabel='Departamento', 
        ylabel='Quantidade',
        title="Médicos por Departamento",
        height=450,          
        width=800,           
        color='#008b8b',     
        line_color='black',  
    ).opts(
        #Inclina os nomes dos departamentos;
        xrotation=45, 
        #Define que o eixo Y deve pular de 0.5 em 0.5 ou 1 em 1
        yticks=[0, 1, 2, 3, 4, 5],
        #Remove a legenda se não forem necessárias;
        show_legend=False,
        #Ativa ferramentas de interatividade na lateral;
        toolbar='right'
    )

aba_grafico = pn.panel(criar_grafico_setores)

In [ ]:
#Nomes, proporções e botões;
btn_add = pn.widgets.Button(name='Incluir', button_type='success', width=150)
btn_upd = pn.widgets.Button(name='Editar', button_type='warning', width=150)
btn_del = pn.widgets.Button(name='Remover', button_type='danger', width=150)
btn_buscar = pn.widgets.Button(name='Buscar', button_type='primary', width=100, align='end')

btn_add.on_click(incluir_medico)
btn_upd.on_click(editar_medico)
btn_del.on_click(remover_medico)
btn_buscar.on_click(carregar_dados)
input_busca.param.watch(carregar_dados, 'value')

In [ ]:
#Eventos e Layout;
aba_gestao = pn.Column(
    "## 🏥 Gestão Hospitalar - Tabela Médico",
    pn.Row(in_crm, in_login, in_dep),
    pn.Row(btn_add, btn_upd, btn_del),
    alerta,
    pn.layout.Divider(),
    "### 🔎 Pesquisar Médico",
    pn.Row(input_busca, btn_buscar),
    pn.Spacer(height=30),
    tabela_visualizacao,
    margin=20
)

app = pn.Tabs(
    ("Gestão de Médicos", aba_gestao),
    ("Relatório", pn.Column("## 📊 Análise por Setor", aba_grafico, margin=20))
)

carregar_dados()
app.servable()